****Ingest Website to Graph DB****

**Part 4** - Use Ve**GenAI Vectorisation** To Collapse Nodes & Relationships

Destroy Entities and Relatonships from Node4J GraphDB using intelligent Vector Search Indices and Queries.

This is adapted from https://python.langchain.com/v0.1/docs/integrations/vectorstores/neo4jvector/ OpenAI Example

Combning it with https://python.langchain.com/v0.1/docs/integrations/text_embedding/google_generative_ai/ 

In [ ]:
%pip install -U langchain langchain-google-genai neo4j json-repair

**Check Version Nos of what was installed**

In [ ]:
%pip show langchain langchain-core langchain-google-genai langchain-google-vertexai langchain-experimental google-ai-generativelanguage langchain-community neo4j google-cloud-aiplatform google-generativeai

**Check Jupyter Version No**

In [ ]:
!jupyter --version

**Check Python Version/Path** - *Expect 3.10.14*

In [ ]:
import sys
import platform
print(sys.version)
print(platform.python_version())
print(sys.path)

**Now for the Imports**

This time we are isloating Vertex AI

In [ ]:
import os

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.documents import Document
from langchain_community.docstore.document import Document
from langchain_community.vectorstores import Neo4jVector

**Diagnostic Methods**

**Connect to Google LLMs**

*Least Privilege Security.*

The Notebook is "owned" by a bespoke Service Account created in terrafrom for this purpose.

Minimal permisisons are added (also via terraform) via predefined roles (esp. Vertex) as required.

This is typically triggered by a PERMISSION DENIED error

In [ ]:
#import getpass
#import os
#from getpass import getpass

#if "GOOGLE_API_KEY" not in os.environ:
#os.environ["GOOGLE_API_KEY"] = getpass("5b049155243b062e344f4dec20fa6520a2e5a6d7")

**GCP Code** corrected as detailed here:

https://www.freecodecamp.org/news/typeerror-module-object-is-not-callable-python-error-solved/

Code below **hangs** because of this issue:

https://stackoverflow.com/questions/24544353/python-getpass-getpass-function-call-hangs#:~:text=getpass()%20will%20freeze%20if,...%20if%20not%20sys.

In [ ]:
import os
# Set It - will require regeneration
os.environ['GOOGLE_API_KEY'] = '3a1bdf0a5c7d4c8faa1be8326e6c9620ba32de58'
# Access the environment variable later in your code
env_api_key = os.environ['GOOGLE_API_KEY']
print(f"env_api_key: {env_api_key}")

## generativelanguage.googleapis.com had to be anabled!

****Enable Langchain Debugging****

See: https://python.langchain.com/v0.1/docs/guides/development/debugging/

In [ ]:
#from langchain.globals import set_debug   
#set_debug(False)

**Enter Node4J**

Node4J Connectivity

Requires singing up for free version.

DB Will be stopped if not recently used and will require resuming else will fail. 

In [ ]:
import os

from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "neo4j+s://a657168d.databases.neo4j.io"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "VM3A9Mz6usNT99nLs_lqQssfVK8JxeD81DnEiXlDkZU"

graph = Neo4jGraph()

**Add to GraphDB**

This statement loads Nodes & Relatonships into Node4J

Thence they can be viewed/manipulated directly on the DB. 

In [ ]:
#graph.add_graph_documents(graph_documents)

**Node4J** already populated by previous Steps

Code Below Initialises a Vector Store from an Existing Graph.

This method pulls relevant text information from the database, and calculates and stores the text embeddings back to the database.

**TBC** Vector Store and Node Vector Index are synonymous. 

In [ ]:
# Vectorisation Parameters - to be populated
gdb_node_label = ""
embedding_model = "models/embedding-001"
gcpEmbedding = GoogleGenerativeAIEmbeddings(model=embedding_model)
gcpEmbedding.google_api_key = "5b049155243b062e344f4dec20fa6520a2e5a6d7"
search_string = ""
test_embedding = "hello, world!"



In [ ]:
vector = gcpEmbedding.embed_query(test_embedding)
vector[:5]

**Try disintermediating langchain**

Example taken from: https://github.com/google-gemini/cookbook/blob/main/examples/Anomaly_detection_with_embeddings.ipynb

In [ ]:
import google.generativeai as genai
import google.ai.generativelanguage as glm

In [ ]:
genai.configure(api_key=env_api_key)

In [ ]:
embedding = genai.embed_content(model=embedding_model,
                                content=test_embedding,
                                task_type="SEMANTIC_SIMILARITY")['embedding']

In [ ]:
# Now we initialize from existing graph
existing_graph = Neo4jVector.from_existing_graph(
    embedding=gcpEmbedding,
    url=os.environ["NEO4J_URI"],
    username=os.environ["NEO4J_USERNAME"],
    password=os.environ["NEO4J_PASSWORD"],
    index_name="person_index",
    node_label=gdb_node_label,
    text_node_properties=["name", "location"],
    embedding_node_property="embedding",
)

**Perform a Search** using the Vector Search Index. 

In [ ]:
result = existing_graph.similarity_search(search_string)
result


In [ ]:
docs_with_score = store.similarity_search_with_score(search_string)
docs_with_score[0]